In [3]:
import datetime
import integration_tasks as integ
import imp

In [4]:
imp.reload(integ)

<module 'integration_tasks' from '/home/kikim/Projects/ai_pipeline/integration_tasks.py'>

In [58]:
class TestPG(integ.PgSelect):
    
    def write(self, rows):
        print(len(rows))
        
        if len(rows) <= 0: return None
        
        columns = self.fill_columns(rows[0], self.columns)
        print(columns)
        
        print(datetime.datetime.now())
        
        dict_list = []
        for i, row in enumerate(rows):
            row_dict = dict(zip(columns, row))
            dict_list.append(row_dict)
            
        print(len(dict_list))
        
        print(dict_list[0])
        print(dict_list[-1])
            

In [59]:
start_time = datetime.datetime.now()
print(start_time)

task = TestPG()
task.columns = ['issued_date', 'logical_path', 'changed_value']
task.sql = "select to_timestamp(PH.issued_date/1000), PD.logical_path, PH.changed_value " + \
            "from point_history PH, point_master PM, point_desc PD " + \
            "where PH.path_id = PM.path_id " + \
            "  and PM.point_path = PD.point_path " + \
            "  and PH.issued_date >= %d " + \
            "  and PH.issued_date < %d " + \
            "order by issued_date, logical_path asc " + \
            ";"
            
task.sql = task.sql % (datetime.datetime(2017,4,1).timestamp() * 1000, 
                       datetime.datetime(2017,4,3).timestamp() * 1000)

print(task.sql)

task.run()

print(datetime.datetime.now() - start_time)

2018-02-14 14:32:21.089079
select to_timestamp(PH.issued_date/1000), PD.logical_path, PH.changed_value from point_history PH, point_master PM, point_desc PD where PH.path_id = PM.path_id   and PM.point_path = PD.point_path   and PH.issued_date >= 1490972400000   and PH.issued_date < 1491145200000 order by issued_date, logical_path asc ;
6061468
['issued_date', 'logical_path', 'changed_value']
2018-02-14 14:33:17.782879
6061468
{'issued_date': datetime.datetime(2017, 4, 1, 0, 0, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=540, name=None)), 'logical_path': '100HP AIR 압력', 'changed_value': '7.306'}
{'issued_date': datetime.datetime(2017, 4, 2, 23, 59, 46, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=540, name=None)), 'logical_path': '응축수탱크_G 펌프_1 상태', 'changed_value': 'Off'}
0:01:05.703634


  ## PostgreSQL 의 LG Point_History를 MongoDB로 옮기기

In [77]:
imp.reload(integ)

<module 'integration_tasks' from '/Users/KIKIM/Projects/ai_pipeline/integration_tasks.py'>

In [ ]:
sql = "select to_timestamp(PH.issued_date/1000), PD.logical_path, PH.changed_value " + \
        "from point_history PH, point_master PM, point_desc PD " + \
        "where PH.path_id = PM.path_id " + \
        "  and PM.point_path = PD.point_path " + \
        "  and PH.issued_date >= %d " + \
        "  and PH.issued_date < %d " + \
        "order by issued_date, logical_path asc " + \
        ";"

start_time = datetime.datetime.now()
print('Start:', start_time)

task = integ.Pg2Mongo()
task.columns = ['issued_date', 'logical_path', 'changed_value']

task.mg_collection = 'point_history'

from_date = datetime.datetime(2013, 7, 1)

while from_date < datetime.datetime.now():
    to_date = from_date + datetime.timedelta(days=10)
    
    task.sql = sql % (from_date.timestamp() * 1000, to_date.timestamp() * 1000)

    print(from_date, '~', to_date, ': ', datetime.datetime.now())
    task.run()
    
    from_date = to_date


# for year in range(2013, 2018):
#     for month in range(1, 13):
        
#         if datetime.datetime(year, month, 1) <= datetime.datetime(2014, 6, 1):
#             continue
        
#         from_date = datetime.datetime(year, month, 1)
        
#         if month < 12:
#             to_date = datetime.datetime(year, month + 1, 1)
#         else:
#             to_date = datetime.datetime(year + 1, 1, 1)
    
print('End:', datetime.datetime.now())
print(datetime.datetime.now() - start_time)

Start: 2018-02-14 20:48:28.229140
2013-07-01 00:00:00 ~ 2013-07-11 00:00:00 :  2018-02-14 20:48:28.229640
No result
2013-07-11 00:00:00 ~ 2013-07-21 00:00:00 :  2018-02-14 20:48:28.389607
77074
2013-07-21 00:00:00 ~ 2013-07-31 00:00:00 :  2018-02-14 20:48:31.025396
880404
2013-07-31 00:00:00 ~ 2013-08-10 00:00:00 :  2018-02-14 20:49:01.606631
1988115
2013-08-10 00:00:00 ~ 2013-08-20 00:00:00 :  2018-02-14 20:50:06.089393
452148
2013-08-20 00:00:00 ~ 2013-08-30 00:00:00 :  2018-02-14 20:50:21.151440
4673743
2013-08-30 00:00:00 ~ 2013-09-09 00:00:00 :  2018-02-14 20:52:40.612785
4796754
2013-09-09 00:00:00 ~ 2013-09-19 00:00:00 :  2018-02-14 20:55:05.877176
4727928
2013-09-19 00:00:00 ~ 2013-09-29 00:00:00 :  2018-02-14 20:57:27.155600
4702353
2013-09-29 00:00:00 ~ 2013-10-09 00:00:00 :  2018-02-14 20:59:47.066746
4689596
2013-10-09 00:00:00 ~ 2013-10-19 00:00:00 :  2018-02-14 21:02:07.374070
4642838
2013-10-19 00:00:00 ~ 2013-10-29 00:00:00 :  2018-02-14 21:04:29.944628
4614286
2013-10-

In [6]:
1+1

2